In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.image as mpimg
import matplotlib.animation as animation

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

import seaborn as sns
import time

import figures

# Trade Matrix Calculated From UN COMTRADE Dataset

The matrices calculated here are created from the publicly available data from UN Comtrade dataset (2001-2009) and Feenstra Dataset (1962-2000). I have collected data on the types of products that are traded between a sample of 70 countries between years 1962 and 2009.

These datasets use Standardized International Trade Codes (SITC), which are used for classify types of goods that are globally traded. SITC codes consist of 4 Digits. Each digit determines a more specific category from left to right. For example: 
* 1 - Beverages and Tobacco 
    * 11 - Beverages
        * 112 - Alcoholic beverages
            * 112.3 - Beer made from malt (including ale, stout, and porter)

Below is a list of the top level codes:
0. Food and live animals
1. Beverages and tobacco
2. Crude materials, inedible, except fuels
3. Mineral, fuels, lubricants and related materials
4. Animal and vegetable oils, fats and waxes
5. Chemicals and related products, n.e.s.
6. Manufactured goods classified chiefly by material
7. Machinery and transport equipment
8. Miscellaeous manufactured articles
9. Commodities and transactions not classified elsewhere

You can find more information regarding SITC Classifications registry at http://unstats.un.org/unsd/cr/registry/regcst.asp?Cl=14

Below is an excerpt from all the data I have concatenated together from these sources

In [ ]:
products_trade = pd.read_csv('data/sitc.csv')
products_trade

Using this dataset I have created role equivalence measure that is previously used in the sociology and management literature. This measure calculates the similarities between nodes in a network by considering the correlations of valued ties. 

To calculate this measure I have first calculated an export product category share vector (EPSV) and an import product category share vector (IPSV) using the formulas below:

### $EPSV_{it} = \frac{exports_{ikt}}{\sum exports_{ikt}}$
### $IPSV_{it} = \frac{imports_{ikt}}{\sum imports_{ikt}}$

I concatenated these two vectors to get a product share vector (PSV) and finally I created a trade matrix based on the correlations of product share vectors for each country.

## Role Equivalence in Trade Matrices for each year:

### $r(PSV_{it}, PSV_{jt})$

Below is the symmetric trade matrix for 1962 where each cell reports for the correlation on the type of products trade between each pair of countries:

In [ ]:
tradeyear1962 = pd.read_csv('data/1962_rev.csv')
tradeyear1962.ix[:, 2:]

In [ ]:
mat = np.array(tradeyear1962.ix[:, 2:])
mat

Below is a dataset from other variables I have collected in my research regarding crises of sovereignty. In this case I have observations regarding the number of revolutionary situations experienced in a country reported by New York Times. This data comes from Banks' Cross National Time Series Dataset. 

In [ ]:
attributevectors1962 = pd.read_csv('data/1962_rev_vector.csv')
attributevectors1962

In [ ]:
rev = np.array(attributevectors1962.ix[:, -1:])
countries = np.array(attributevectors1962.ix[:, 1:2])
countries

To visualize this data in a meaningful way, I have used the manifold learning algorithms for metric multidimensional scaling from scikit-learn module in Python. Multidimensional scaling allows a two dimensional representation of the data by respecting the original distances of the high-dimensional space. In this case for my graphs, countries that have a higher correlation of trade products (stronger ties) are represented closer to each other. Countries that have a lower correlation are farther apart from each other. This is a nice way of visually representing this data because for my work my hypothesis that countries that have higher similarities in their trade behavior are more likely to experience crises in similar time periods

In [ ]:
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=2)
mds

Below is the coordinates for the countries calculated for 1962:

In [ ]:
results = mds.fit_transform(mat)
results

Using matplotlib I created the figure below. In the figure, I color coded the countries that experienced a revolutionary situation with black dots. Other countries have white dots. I expect black dots to be in general closer to each other. I also expect that in the following year, the countries that are close to black dots should be more likely to experience a revolutionary situation 

In [ ]:
fig = plt.figure(figsize=(28.0,16.5))
fig.suptitle('Revolutionary Situations in 1962', fontsize=14, fontweight='bold')


plt.subplots_adjust(bottom = 0.1)

plt.scatter(
    results[:, 0], results[:, 1], marker = 'o', c=rev
    )


for label, x, y, in zip(countries, results[:, 0], results[:, 1]):
    plt.annotate(
    label,
    xy = (x, y), xytext = (-20, 20),
    textcoords = 'offset points', ha = 'right', va='bottom',
    bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
    arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

leg = plt.legend(('Revolutions', 'Stable'), loc = 'lower right', frameon=True)    
leg.get_frame().set_edgecolor('b')
plt.show()

Elsewhere I have written a program that has functions to automate the commands I have used above. With the loop below I have generated a plot for each year in my dataset

In [ ]:
year = 1962
while year<2010:
    numbers = figures.inputs('data/'+str(year)+'_rev.csv', 'data/'+str(year)+'_rev_vector.csv')
    coor = figures.mdscoordinates(numbers['matrix'])
    figures.mdsfigure(coor, year, numbers['behavior'], numbers['country names'])
    year+=1
#    plt.pause(0.5)
    plt.close('all')

Below is a plot that displays how countries move across the graph from 1962 to 2009:

In [ ]:
year = 1962
while year<2010:
    plt.figure(figsize=(28.0, 16.5))
    img=mpimg.imread('images/'+str(year)+'.png')
    imgplot = plt.imshow(img)
    year+=1
    plt.pause(1)
    plt.close('all')